In [63]:
%matplotlib inline
from IPython.display import display, Math
import sympy as sp
import numpy as np
from sympy import init_printing
init_printing() # doctest: +SKIP
from mueller_matrices import M_Retarder

## Sample model: linear retarder with retardance $\delta$ and fast axis $\theta$
#### Slow axis is $\phi = \theta + \pi/2$

In [48]:
delta, phi = sp.symbols('delta, phi')
M_sample = M_Retarder(phi - sp.pi/2, delta)

M_sample

⎡1                 0                                0                        0
⎢                                                                             
⎢       2                  2                                                  
⎢0   sin (2⋅φ)⋅cos(δ) + cos (2⋅φ)    (-cos(δ) + 1)⋅sin(2⋅φ)⋅cos(2⋅φ)  sin(δ)⋅s
⎢                                                                             
⎢                                        2                  2                 
⎢0  (-cos(δ) + 1)⋅sin(2⋅φ)⋅cos(2⋅φ)   sin (2⋅φ) + cos(δ)⋅cos (2⋅φ)    -sin(δ)⋅
⎢                                                                             
⎣0         -sin(δ)⋅sin(2⋅φ)                  sin(δ)⋅cos(2⋅φ)               cos

        ⎤
        ⎥
        ⎥
in(2⋅φ) ⎥
        ⎥
        ⎥
cos(2⋅φ)⎥
        ⎥
(δ)     ⎦

## Sample may also have transmission $t$ and degree of polarization $p$

In [49]:
t, p = sp.symbols('t, p')
M_tp = sp.Matrix([[t, 0, 0, 0], [0, t*p, 0, 0], [0, 0, t*p, 0], [0, 0, 0, t*p]])
M_sample = M_tp * M_sample

M_sample

⎡t                   0                                    0                   
⎢                                                                             
⎢       ⎛   2                  2     ⎞                                        
⎢0  p⋅t⋅⎝sin (2⋅φ)⋅cos(δ) + cos (2⋅φ)⎠   p⋅t⋅(-cos(δ) + 1)⋅sin(2⋅φ)⋅cos(2⋅φ)  
⎢                                                                             
⎢                                            ⎛   2                  2     ⎞   
⎢0  p⋅t⋅(-cos(δ) + 1)⋅sin(2⋅φ)⋅cos(2⋅φ)  p⋅t⋅⎝sin (2⋅φ) + cos(δ)⋅cos (2⋅φ)⎠   
⎢                                                                             
⎣0         -p⋅t⋅sin(δ)⋅sin(2⋅φ)                  p⋅t⋅sin(δ)⋅cos(2⋅φ)          

         0          ⎤
                    ⎥
                    ⎥
p⋅t⋅sin(δ)⋅sin(2⋅φ) ⎥
                    ⎥
                    ⎥
-p⋅t⋅sin(δ)⋅cos(2⋅φ)⎥
                    ⎥
     p⋅t⋅cos(δ)     ⎦

## Stokes vector of excitation light

In [108]:
S_unp = sp.Matrix([[1, 0, 0, 0]]).T
S_rcp = sp.Matrix([[1, 0, 0, 1]]).T
S_lcp = sp.Matrix([[1, 0, 0, -1]]).T
S = sp.Matrix([sp.symbols('s0:4')]).T
S_in = sp.Matrix([sp.symbols('s_0:4^{in}')]).T
S_out = sp.Matrix([sp.symbols('s_0:4^{out}')]).T

## Model 1: Excitation using RCP light

In [51]:
S_sample = M_sample * S_rcp

S_sample

⎡         t          ⎤
⎢                    ⎥
⎢p⋅t⋅sin(δ)⋅sin(2⋅φ) ⎥
⎢                    ⎥
⎢-p⋅t⋅sin(δ)⋅cos(2⋅φ)⎥
⎢                    ⎥
⎣     p⋅t⋅cos(δ)     ⎦

## The microscope measures the stokes vector of ligth $\vec s = \left< s_0, s_1, s_2, s_3 \right>$ after interacting with the sample

In [136]:
transmission = S[0]
slow_axis = 0.5 * sp.atan(-S[1]/S[2])
retardance = sp.atan(sp.sqrt(S[1]**2+S[2]**2)/S[3])
polarization = sp.sqrt(S[1]**2+S[2]**2+S[3]**2)/S[0]

display(Math('transmission = '+sp.latex(transmission)))
display(Math('slow\, axis = '+sp.latex(slow_axis)))
display(Math('retardance = '+sp.latex(retardance)))
display(Math('polarization = '+sp.latex(polarization)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## General model

In [109]:
S_measured = M_sample * S_in

display(Math(sp.latex(S_out)+'='+sp.latex(M_sample * S_in)))

<IPython.core.display.Math object>

### If S_in = S_rcp

In [114]:
display(Math(sp.latex(S_out)+'='+sp.latex(S_measured.subs({'s_0^{in}':1,'s_1^{in}':0,'s_2^{in}':0,'s_3^{in}':1}))))

<IPython.core.display.Math object>

### If s1_in and s2_in are non-zero, but small; if p $\approx$ 1

#### If retardance delta is small, take Taylor expansion of cos(delta) to first term

$cos(\delta)=1-\frac{x^2}{2!}+\frac{x^4}{4!}+...$

In [134]:
S2 = S_measured.subs({'s_0^{in}':1,sp.cos(delta):1,p:1})
display(Math(sp.latex(S_out)+'='+sp.latex(sp.trigsimp(S2))))

<IPython.core.display.Math object>

### $t=\frac{s_0^{out}}{s_0^{in}}$

### $\phi=\frac{1}{2}\arctan(-\frac{s_1^{norm}-s_1^{in}}{s_2^{norm}-s_2^{in}})$

### $\delta=\arcsin(\frac{\sqrt{(s_1^{norm}-s_1^{in})^2+(s_2^{norm}-s_2^{in})^2}}{s_3^{in}})$

#### Depolarization affects the accuracy of background correction

#### Full equations can be solved without assuming small retardance and depolarization